In [25]:
%use kandy, multik

In [26]:
import kotlin.random.Random
import kotlin.math.PI
import org.apache.commons.math3.distribution.PoissonDistribution

In [27]:
import kotlin.math.*

data class Point(
    val x: Double,
    val y: Double
) {

    constructor(x: Number, y: Number) : this(x.toDouble(), y.toDouble())

    fun distTo(other: Point): Double {
        return sqrt((x - other.x).pow(2) + (y - other.y).pow(2))
    }

    fun middleTo(other: Point): Point {
        return Point((x + other.x) / 2, (y + other.y) / 2)
    }

    fun adjust(dx: Number, dy: Number): Point {
        return Point(x + dx.toDouble(), y + dy.toDouble())
    }

    fun adjust(adjustment: Adjustment): Point {
        return adjust(adjustment.x, adjustment.y)
    }

    fun getAdjustment(to: Point): Adjustment {
        return Adjustment(to.x - x, to.y - y)
    }

    fun isInSquare(size: Number): Boolean {
        return x in 0.0..size.toDouble() && y in 0.0..size.toDouble()
    }

    fun isInCircle(center: Point, radius: Number): Boolean {
        return distTo(center) <= radius.toDouble()
    }

    fun interpolate(other: Point): List<Point> {
        return interpolate(other, 1000)
    }

    fun interpolate(other: Point, n: Int): List<Point> {
        val res = ArrayList<Point>()
        val a = getAdjustment(other)
        val b = Adjustment(a.x / (n + 1), a.y / (n + 1))
        var p = this
        repeat(n) {
            val t = p.adjust(b)
            res.add(t)
            p = t
        }
        return res
    }
}

data class Adjustment(
    val x: Double,
    val y: Double
) {
    constructor(x: Number, y: Number) : this(x.toDouble(), y.toDouble())
}

private val TWO_PI = 2 * PI

data class Sector(
    val pivot: Point,
    val direction: Double,
    val fov: Double
) {
    private val boundUpper = direction + fov / 2
    private val boundLower = direction - fov / 2

    fun getDirectionEndPoint(distance: Number): Point {
        return pointFromPivotWithAngle(distance, direction)
    }

    private fun pointFromPivotWithAngle(distance: Number, directionAngle: Double): Point {
        val x = cos(directionAngle) * distance.toDouble()
        val y = sin(directionAngle) * distance.toDouble()
        return Point(pivot.x + x, pivot.y + y)
    }

    fun contains(p: Point): Boolean {
//        println("Sector: $this")
//        println("Other: $p")

        val adjustment = pivot.getAdjustment(p)
        val angle = atan2(adjustment.y, adjustment.x).let {
            if (it < 0) it + TWO_PI else
                if (it > TWO_PI) it - TWO_PI else it
        }
//        println("Angle: ${degrees(angle)}")
        return angle in boundLower..boundUpper
    }

    fun getLineEndsInSquare(distance: Number): Pair<Point, Point> {
        return Pair(
            pointFromPivotWithAngle(distance, boundUpper),
            pointFromPivotWithAngle(distance, boundLower)
        )
    }

    override fun toString(): String {
        return "Sector(pivot=$pivot, direction=${degrees(direction)}, fov=${degrees(fov)}, bu=${degrees(boundUpper)}, bl=${degrees(boundLower)})"
    }

    private fun degrees(angle: Number): Int {
        return ((angle.toDouble() / TWO_PI) * 360).toInt()
    }
}

In [28]:
import kotlin.math.PI
import kotlin.math.cos
import kotlin.math.sin
import kotlin.math.sqrt
import kotlin.random.Random

object Generator {

    fun nextPointInRectangle(width: Number, height: Number): Point {
        return Point(
            Random.nextDouble(0.0, width.toDouble()),
            Random.nextDouble(0.0, height.toDouble())
        )
    }

    fun nextPointInSquare(size: Number): Point {
        return nextPointInRectangle(size, size)
    }

    fun nextPointInCircleAsSquare(radius: Number = 1.0): Point {
        return nextPointInCircleAsSquare(radius.toDouble())
    }

    private fun nextPointInCircleAsSquare(radius: Double = 1.0): Point {
        val zeroPoint = Point(0.0, 0.0)
        var point: Point
        do {
            point = Point(
                Random.nextDouble(-radius, radius),
                Random.nextDouble(-radius, radius),
            )
        } while (point.distTo(zeroPoint) > radius)
        return point
    }

    fun nextPointInCircle(maxRadius: Number = 0.5): Point {
        val angle = Random.nextDouble() * 2 * PI
        val radius = sqrt(Random.nextDouble()) * maxRadius.toDouble()
        return Point(radius * cos(angle), radius * sin(angle))
    }

    fun nextOnDistance(point: Point, distance: Number): Point {
        val angle = Random.nextDouble() * 2 * PI
        return Point(point.x + distance.toDouble() * cos(angle), point.y + distance.toDouble() * sin(angle))
    }

    fun nextPariInSquare(distance: Number, squareSize: Number): Pair<Point, Point> {
        var first = nextPointInSquare(squareSize)
        var second = nextOnDistance(first, distance)
        if (!(first.isInSquare(squareSize) && second.isInSquare(squareSize))) {
            val squareCenter = Point(squareSize.toDouble() / 2, squareSize.toDouble() / 2)
            val middle = first.middleTo(second)
            val adjustment = middle.getAdjustment(squareCenter)
            first = first.adjust(adjustment)
            second = second.adjust(adjustment)
        }
        return Pair(first, second)
    }
}


In [50]:
val rand = Random(1337)

val R = 10.0
val circleArea = PI * R * R

val A = 0.15
val B = 0.15
val D = 0.1
val fov = PI / 2

val receiverPoint = Point(0, 0)

val bv = circleArea * B
val av = circleArea * A

println(bv)
println(av)

val interfereObjectDistribution = PoissonDistribution(bv)
val blockObjectDistribution =  PoissonDistribution(av)

47.1238898038469
47.1238898038469


In [51]:
val interfereObjects = HashSet<Sector>()

repeat(interfereObjectDistribution.sample()) {
    interfereObjects.add(Sector(Generator.nextPointInCircle(R), rand.nextDouble(0.0, 2 * PI), fov))
}

val blockObjects = ArrayList<Point>()

repeat(blockObjectDistribution.sample()) {
    blockObjects.add(Generator.nextPointInCircle(R))
}

In [52]:
println("${interfereObjects.size} ${blockObjects.size}")

50 47


In [53]:
fun fspl(distance: Number, frequency: Number): Double {
    return 20.0 * log(distance.toDouble(), 10.0) + 20.0 * log(frequency.toDouble(), 10.0) - 147.55
}

In [56]:
var interfereCounter = 0

val f = 1_000_000_000

val interGain = ArrayList<Double>()

val gain = 2.0 / (1 - cos(fov / 2.0))
val logGain = 10.0 * log(gain, 10.0)

println(gain)

for (sector in interfereObjects) {
    if (sector.contains(receiverPoint)) {
        var rejected = false
        interp@for (i in sector.pivot.interpolate(receiverPoint)) {
            for (block in blockObjects) {
                if (i.isInCircle(block, D)) {
                    rejected = true
                    break@interp
                }
            }
        }
        if (!rejected) {
            ++interfereCounter
            val fspl = fspl(sector.pivot.distTo(receiverPoint), f)
            interGain.add(fspl + logGain)
        }
    }
}

println(interfereCounter)
println(interGain.sum())

6.828427124746192
9
501.6893975876351


In [55]:
import org.jetbrains.letsPlot.Geom

plot {
    points {
        x(interfereObjects.map{ it.pivot.x })
        y(interfereObjects.map{ it.pivot.y })
        color = Color.WHITE
    }
    points {
        x(blockObjects.map{ it.x })
        y(blockObjects.map{ it.y })
        color = Color.GREEN
        size = D * 50
    }
    points {
        x(listOf(receiverPoint.x))
        y(listOf(receiverPoint.y))
        color = Color.RED
    }
    interfereObjects.forEach { s ->
        val de = s.getDirectionEndPoint(1)
        val le = s.getLineEndsInSquare(1)
        line {
            x(listOf(s.pivot.x, le.first.x))
            y(listOf(s.pivot.y, le.first.y))
            color = Color.GREY
        }
        line {
            x(listOf(s.pivot.x, le.second.x))
            y(listOf(s.pivot.y, le.second.y))
            color = Color.GREY
        }
        line {
            x(listOf(s.pivot.x, de.x))
            y(listOf(s.pivot.y, de.y))
            color = Color.WHITE
        }
    }
    layout {
        size = 500 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gp3NEY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":500.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,nu

In [46]:
fun interference(fov: Double, B: Double, A: Double): Double {

    val R = 10.0
    val circleArea = PI * R * R

    val D = 0.1

    val receiverPoint = Point(0, 0)

    val bv = circleArea * B
    val av = circleArea * A

    val interfereObjectDistribution = PoissonDistribution(bv)
    val blockObjectDistribution =  PoissonDistribution(av)

    val results = ArrayList<Double>()
    val f = 1_000_000_000
    val gain = 2.0 / (1 - cos(fov / 2.0))
    val logGain = 10.0 * log(gain, 10.0)

    repeat(1000) {
        val interfereObjects = HashSet<Sector>()

        repeat(interfereObjectDistribution.sample()) {
            interfereObjects.add(Sector(Generator.nextPointInCircle(R), rand.nextDouble(0.0, 2 * PI), fov))
        }

        val blockObjects = ArrayList<Point>()

        repeat(blockObjectDistribution.sample()) {
            blockObjects.add(Generator.nextPointInCircle(R))
        }

        var interfereCounter = 0

        val interGain = ArrayList<Double>()

        for (sector in interfereObjects) {
            if (sector.contains(receiverPoint)) {
                var rejected = false
                interp@for (i in sector.pivot.interpolate(receiverPoint)) {
                    for (block in blockObjects) {
                        if (i.isInCircle(block, D)) {
                            rejected = true
                            break@interp
                        }
                    }
                }
                if (!rejected) {
                    ++interfereCounter
                    val fspl = fspl(sector.pivot.distTo(receiverPoint), f)
                    interGain.add(fspl + logGain)
                }
            }
        }

        results.add(interGain.sum())
    }

    return results.average()
}

In [47]:
val fovs = (1..180).map { it * PI / 180 }

val values = fovs.map { interference(it, 0.05, 0.05) }

plot {
    line {
        x(fovs)
        y(values)
    }
    layout {
        xAxisLabel = "fov, rad"
        yAxisLabel = "interference, dBm"
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="CLUAS7"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"fov, rad"
},{
"aesthetic":"y",
"name":"interference, dBm"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.017453292519943295,0.03490658503988659,0.05235987755982988,0.06981317007977318,0.08726646259971647,0.10471975511965977,0.12217304763960307,0.13962634015954636,0.15707963267948966,0.17453292519943295,0.19198621771937624,0.20943951023931953,0.22689280275926285,0.24434609527920614,0.2617993877991494,0.2792526803190927,0.29670597283903605,0.3141592653589793,0.3316125578789226,0.3490658503988659,0.3665191429188092,0.3839724354387525,0.40142572795869574,0.41887902047863906,0.4363323129985824,0.4537856055185257,0.47123889803846897,0.4886921905584123,0.5061454830783556,0.5235987755982988,0.5410520681182421,0.5585053606381855,0.5759586531581288,0.5934119456780721,0.6108652381980153,0.6283185307179586,0.6457718232379019,0.6632251157578452,0.6806784082777885,0.6981317007977318,0.715584993317675,0.7330382858376184,0.7504915783575616,0.767944870877505,0.7853981633974483,0.8028514559173915,0.8203047484373349,0.8377580409572781,0.8552113334772214,0.8726646259971648,0.890117918517108,0.9075712110370514,0.9250245035569946,0.9424777960769379,0.9599310885968813,0.9773843811168246,0.9948376736367678,1.0122909661567112,1.0297442586766543,1.0471975511965976,1.064650843716541,1.0821041362364843,1.0995574287564276,1.117010721276371,1.1344640137963142,1.1519173063162575,1.1693705988362006,1.1868238913561442,1.2042771838760873,1.2217304763960306,1.239183768915974,1.2566370614359172,1.2740903539558606,1.2915436464758039,1.3089969389957472,1.3264502315156903,1.3439035240356338,1.361356816555577,1.3788101090755203,1.3962634015954636,1.413716694115407,1.43116998663535,1.4486232791552935,1.4660765716752369,1.4835298641951802,1.5009831567151233,1.5184364492350666,1.53588974175501,1.5533430342749535,1.5707963267948966,1.5882496193148399,1.605702911834783,1.6231562043547263,1.6406094968746698,1.6580627893946132,1.6755160819145563,1.6929693744344996,1.710422666954443,1.7278759594743864,1.7453292519943295,1.7627825445142729,1.780235837034216,1.7976891295541593,1.8151424220741028,1.8325957145940461,1.8500490071139892,1.8675022996339325,1.8849555921538759,1.902408884673819,1.9198621771937625,1.9373154697137058,1.9547687622336491,1.9722220547535922,1.9896753472735356,2.007128639793479,2.0245819323134224,2.0420352248333655,2.0594885173533086,2.076941809873252,2.0943951023931953,2.111848394913139,2.129301687433082,2.1467549799530254,2.1642082724729685,2.1816615649929116,2.199114857512855,2.2165681500327987,2.234021442552742,2.251474735072685,2.2689280275926285,2.286381320112572,2.303834612632515,2.321287905152458,2.3387411976724013,2.356194490192345,2.3736477827122884,2.3911010752322315,2.4085543677521746,2.426007660272118,2.443460952792061,2.4609142453120043,2.478367537831948,2.4958208303518914,2.5132741228718345,2.5307274153917776,2.548180707911721,2.5656340004316647,2.5830872929516078,2.600540585471551,2.6179938779914944,2.6354471705114375,2.6529004630313806,2.670353755551324,2.6878070480712677,2.705260340591211,2.722713633111154,2.740166925631097,2.7576202181510405,2.775073510670984,2.792526803190927,2.8099800957108703,2.827433388230814,2.844886680750757,2.8623399732707,2.8797932657906435,2.897246558310587,2.9146998508305306,2.9321531433504737,2.949606435870417,2.9670597283903604,2.9845130209103035,3.0019663134302466,3.01941960595019,3.036872898470133,3.0543261909900763,3.07177948351002,3.0

In [48]:
val interferingObjects = (1..10).map { it / 100.0}

val values = interferingObjects.map { interference(PI / 180 * 45, it, 0.05) }

plot {
    line {
        x(interferingObjects)
        y(values)
    }
    layout {
        xAxisLabel = "interfering objects rate"
        yAxisLabel = "interference, dBm"
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cvSIPV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"interfering objects rate"
},{
"aesthetic":"y",
"name":"interference, dBm"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1],
"y":[21.90199590710819,44.190531099800644,65.57696602880569,87.91996717881142,109.00436100910152,132.44292494500078,160.76986684305692,172.3256354452861,198.89358980019222,214.31165008810694]
},
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotContainer = document.getElementById("cvSIPV");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 
 
 
 
 
 
 0.02 
 
 
 
 
 
 
 
 
 0.03 
 
 
 
 
 
 
 
 
 0.04 
 
 
 
 
 
 
 
 
 0.05 
 
 
 
 
 
 
 
 
 0.06 
 
 
 
 
 
 
 
 
 0.07 
 
 
 
 
 
 
 
 
 0.08 
 
 
 
 
 
 
 
 
 0.09 
 
 
 
 
 
 
 
 
 0.10 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 interference, dBm 
 
 
 
 
 interfering objects rate

In [49]:
val blockingObjects = (1..20).map { it / 100.0}

val values = blockingObjects.map { interference(PI / 180 * 45, 0.10, it) }

plot {
    line {
        x(blockingObjects)
        y(values)
    }
    layout {
        xAxisLabel = "blocking objects rate"
        yAxisLabel = "interference, dBm"
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="5dyyrG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"blocking objects rate"
},{
"aesthetic":"y",
"name":"interference, dBm"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2],
"y":[244.45253809750966,225.0061345590221,230.17461661509512,226.2721823448059,220.3757301622606,218.2095322177981,216.46474659108435,212.330433672307,210.06416685725384,211.7375898399522,197.10677029378957,203.53397854247876,196.65727022571286,194.54109744987744,186.74025171144729,190.6041230394047,186.0166724518417,188.03055895826472,187.26200220401074,177.07850283877033]
},
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotContainer = document.getElementById("5dyyrG");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.275371983345245 15.5 L24.275371983345245 15.5 L49.82839512370865 104.976384690038 L75.38141826407207 81.19527329854213 L100.93444140443547 99.15107213326814 L126.4874645447989 126.2817044246143 L152.0404876851623 136.2487690394098 L177.59351082552573 144.27684012353131 L203.14653396588912 163.2995541393841 L228.6995571062525 173.72705354577226 L254.25258024661596 166.02733476472747 L279.8056033869794 233.3463552433485 L305.3586265273428 203.77361925622802 L330.9116496677062 235.41458554258247 L356.4646728080696 245.15147539918178 L382.017695948433 281.0445634356396 L407.5707190887964 263.26619960353673 L433.12374222915986 284.3738811708221 L458.6767653695232 275.1076287548343 L484.22978850988665 278.6438959964173 L509.7828116502501 325.4999999999998 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.02 
 
 
 
 
 
 
 
 
 0.04 
 
 
 
 
 
 
 
 
 0.06 
 
 
 
 
 
 
 
 
 0.08 
 
 
 
 
 
 
 
 
 0.10 
 
 
 
 
 
 
 
 
 0.12 
 
 
 
 
 
 
 
 
 0.14 
 
 
 
 
 
 
 
 
 0.16 
 
 
 
 
 
 
 
 
 0.18 
 
 
 
 
 
 
 
 
 0.20 
 
 
 
 
 
 
 
 
 
 
 180 
 
 
 
 
 
 
 190 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 210 
 
 
 
 
 
 
 220 
 
 
 
 
 
 
 230 
 
 
 
 
 
 
 240 
 
 
 
 
 
 
 
 
 interference, dBm 
 
 
 
 
 blocking objects rate